In [7]:
import pandas as pd

In [21]:
annual = pd.read_csv("C:/Users/user1/Desktop/AnnualAnalysis.csv", encoding='gb18030')
annual = annual[['ddate', 'prdlinename', 'sub_plan', 'revenue']]
annual.head(10)



,ddate,prdlinename,sub_plan,revenue
0,202301.0,xlab,mon_plan,37.3849
1,202301.0,xlab,quar_plan,0.104
2,202301.0,xlab,yr_plan,60.1364
3,202301.0,public1,mon_plan,284.1489303
4,202301.0,public1,quar_plan,12.8544
5,202301.0,public1,half_yr,70.024
6,202301.0,public1,yr_plan,"1,595.99"
7,202301.0,public2,mon_plan,0.85986317
8,202301.0,public2,quar_plan,4.2848
9,202301.0,public2,yr_plan,73.540875
